# Test Your Model
The code below illustrates how to load the model trained and saved at notebook *cw2-train.ipynb*, and test the performance of the loaded model with held-out test data. 

**REMARK 1**: You should adjust the code below according to what components you have saved and how you have saved them.

**REMARK 2**: When the markers evaluate your model, they will use the code below (but replace the test data with some held-out data) to run the test. **Hence, make sure you can re-load your model and test its performance with the code below.**

**REMARK 3**: If you use embeddings to represent text, **DO NOT** include the embeddings file in your submitted file due to its huge size. Instead, specify which pre-trained embedding you want to use or provide a link for downloading it; the markers will download the embedding and run your code below to load the embeddings.



In [3]:
#from google.colab import drive
#drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# NOTE! The model defined below MUST BE EXACTLY THE SAME as the one you used at training

from nltk.tokenize import word_tokenize
import numpy as np
import torch
import torch.nn as nn


class CNN_Clf(nn.Module):
    def __init__(self, embd_dim, filter_size_list, filter_num_list, class_num, dp_rate=0.5, gpu=False):
        super(CNN_Clf, self).__init__()
        self.embd_dim = embd_dim
        assert len(filter_size_list) == len(filter_num_list)
        self.output_dim = class_num
        self.tanh = nn.Tanh()
        self.dropout = nn.Dropout(dp_rate)
        self.fc = nn.Linear(np.sum(filter_num_list), class_num)
        self.gpu = gpu
        self.convs = self.build_convs(filter_size_list, filter_num_list, gpu)
        if self.gpu:
            self.to('cuda')
            
    def build_convs(self, f_sizes, f_nums, gpu):
        convs = nn.ModuleList()
        for fs, fn in zip(f_sizes, f_nums):
            padding_size = fs-1
            m = nn.Conv1d(self.embd_dim, fn, fs, padding=padding_size)
            if gpu: m.to('cuda')
            convs.append(m)
        return convs
        
    def get_conv_output(self, input_matrix, conv, gpu):
        # step 1: compute convolution 
        assert input_matrix.shape[1] == self.embd_dim
        conv_output = conv(input_matrix)
        # step 2: pass through an activation function 
        conv_relu = self.tanh(conv_output)
        # step 3: max-over-time pooling
        maxp = nn.MaxPool1d(conv_relu.shape[2])
        maxp_output = maxp(conv_relu)
        return maxp_output
       
    def forward(self, all_text_vectors):
        cnn_repr = torch.tensor([])
        if self.gpu: cnn_repr = cnn_repr.to('cuda')
        for cv in self.convs:
            cv_output = self.get_conv_output(all_text_vectors, cv, self.gpu)
            cnn_repr = torch.cat((cnn_repr, cv_output), dim=1)
        # print(cnn_repr.shape)
        after_dp = self.dropout(cnn_repr.squeeze())
        logit = self.fc(after_dp)
        # the CrossEntropyLoss provided by pytorch includes softmax; so you do not need to include a softmax layer in your net
        return logit
#word_vectors['hello']

In [5]:
# if you use word embeddings to represent text, provide the script for loading the embeddings below
# if you use glove, word2vec or fasttext embeddings, please specify which version you use (e.g. glove.6B.300d)
# if you use other embedding models, please provide the download link

import pandas as pd
from sklearn.utils import shuffle
import pandas as pd
from sklearn.utils import shuffle
import nltk
nltk.download('punkt')
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

# load pre-trained glove embeddings
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

# specify the loaction of the downloaded glove file
path_of_downloaded_files = "/content/drive/My Drive/Colab Notebooks/glove.6B.300d.txt"
glove_file = datapath(path_of_downloaded_files)
word2vec_glove_file = get_tmpfile("glove.6B.300d.txt")
glove2word2vec(glove_file, word2vec_glove_file)
word_vectors = KeyedVectors.load_word2vec_format(word2vec_glove_file)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
# scripts for creating sentence vectors; adjust the code if necessary

from nltk.tokenize import word_tokenize
def vectorize_sent(word_vectors, sent, oov_vec):
    word_vecs = []
    for token in word_tokenize(sent): 
        if token not in word_vectors: 
            word_vecs.append(oov_vec)
        else:
            word_vecs.append(word_vectors[token].astype('float64'))
    return np.mean(word_vecs,axis=0)

In [0]:
# reconstruct your trained model from pickle

import pickle
def reconstruct_model(pickle_path):
    saved_model_dic = pickle.load(open(pickle_path,"rb"))
    input_dim = saved_model_dic['input_dim']
    dp_rate = saved_model_dic['dropout_rate']
    filter_size_list = saved_model_dic['filter_size_list']
    filter_num_list = saved_model_dic['filter_num_list']
    output_dim = 2
    class_num = saved_model_dic['class_num']
    oov_vec = saved_model_dic['oov_vector']
    model = CNN_Clf(input_dim, filter_size_list, filter_num_list, class_num)
    saved_weights = saved_model_dic['neural_weights']
    model.load_state_dict(saved_weights)

    
    return model, oov_vec

In [0]:
# use the reconstructed model to make predictions on the test data

# use the reconstructed model to make predictions on the test data
#oov_vec  = saved_model_dic['oov_vector']
def get_sent_word_vecs(word_vectors, sent_words, largest_len, oov_vec):
    word_vec_dim = 300
    vecs = []
    for ww in sent_words:
        if ww in word_vectors:
            vecs.append(word_vectors[ww])
        else:
            vecs.append(oov_vec)
    for i in range(largest_len-len(sent_words)):
        vecs.append([0.]*word_vec_dim)
    return np.array(np.transpose(vecs))


def build_mini_batch(sent_list, word_vectors, oov_vec):
    #oov_vec  = saved_model_dic['oov_vector']
    tokenized_sents = [word_tokenize(ss.lower()) for ss in sent_list]
    largest_len = np.max([len(tokens) for tokens in tokenized_sents])
    text_vecs = []
    for ts in tokenized_sents:
        vv = get_sent_word_vecs(word_vectors, ts, largest_len, oov_vec)
        text_vecs.append(vv)
    # print('mini batch shape',np.array(text_vecs).shape)
    return np.array(text_vecs)

def test_trained_model(model, oov_vec, test_text):
    #oov_vec  = saved_model_dic['oov_vector']
    test_vecs = build_mini_batch(test_text, word_vectors, oov_vec)
    test_vecs_tensor = torch.tensor(test_vecs, dtype=torch.float)
    test_prediction = model(test_vecs_tensor)
    pred_labels = [np.argmax(tp.detach().numpy()) for tp in test_prediction]
    return pred_labels

In [39]:
# load sample test data
import pandas as pd
import numpy as np
import re 

# df = pd.read_table('/content/drive/My Drive/Colab Notebooks/coursework2_train.tsv')
# df = shuffle(df) 
test_data = pd.read_table('/content/drive/My Drive/Colab Notebooks/coursework2_train.tsv')
#test_data = shuffle(test_data)
test_text = test_data['sentence_text'].tolist()[-2000:]
test_titles = test_data['article_title'].tolist()[-2000:]
test_raw_labels = test_data['label'].tolist()[-2000:]
label_dic = {'non-propaganda':0, 'propaganda':1} 
test_labels = [label_dic[rl] for rl in test_raw_labels]

print('test data size', len(test_labels))


import string

combined = []
for tits, dics in zip(test_titles, test_text):
    combined.append(tits + " " + dics)
#combined2 = [" ".join(ss) for ss in combined]

combined3 = []
for sent in combined:
    combined3.append(" ".join(re.findall('\w+', sent.lower())))

test_input = combined3

# reconstruct model and make predictions
model, oov_vec = reconstruct_model('/content/drive/My Drive/Colab Notebooks/cw2_sample_saved_file.pickle')
test_pred = test_trained_model(model, oov_vec, test_input)

# test model
from sklearn.metrics import precision_recall_fscore_support,accuracy_score
pre, rec, f1, _ = precision_recall_fscore_support(test_labels, test_pred, average='macro')
print('macro-F1 on test data', f1)

test data size 2000
macro-F1 on test data 0.9739348996842183


In [2]:
combined3

NameError: ignored

In [41]:
test_data[100:110]['sentence_text'].tolist()

['Hinson “told the students that they could do these activities if they wanted,” Branch claimed.',
 '“The teacher has told her class several times that this is a study of world religions and that she is not trying to advocate for any religion over another.',
 'She has told her class that if they had questions about religious beliefs, that those conversations should take place with their parents,” Branch added.',
 'Penkoski claimed the teacher sent students home with the same packet the day after he lodged a complaint — this time, with certain sections crossed out but still including the Shahada assignment.',
 'Penkoksi called the principal again and confronted Hinson over the phone.',
 '“I said, ‘This is not OK in asking my kid to write down the Shahada.’ The teacher happened to walk-in and said she made it an option and that the kids didn’t have to do it.',
 'My daughter conflicted that story and said, ‘No, that is not what was said.’ What was said was, ‘Do the assignment; and if you 